<a href="https://colab.research.google.com/github/santhoshraj10/Generating-Keywords-for-Google-Ads/blob/master/Capstone_Google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json 
from pandas.io.json import json_normalize
import datetime
pd.options.display.max_rows
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#df=pd.read_csv('/content/drive/MyDrive/Capstone/train 5.csv')
json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train 5.csv', converters={column: json.loads for column in json_cols})
for column in json_cols:
    column_as_df = json_normalize(df[column])
    column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
    df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
#df['fullVisitorId'] = df['fullVisitorId'].astype('str')

In [ ]:
df.head()

In [ ]:
!nvidia-smi

In [ ]:
df.shape

In [ ]:
df["totals_transactionRevenue"].fillna(0,inplace=True)

In [ ]:
df.totals_transactionRevenue.value_counts()

In [ ]:
for i in range(df.shape[0]):
    if df.totals_transactionRevenue.iloc[i]!=0:
        df.totals_transactionRevenue.iloc[i]=1

In [ ]:
df.totals_transactionRevenue.value_counts()

In [ ]:
null_values=df.isnull().sum()/len(df)*100
null_values

In [ ]:
null_value_Feat=null_values[null_values.values>=50]

In [ ]:
null_value_Feat

In [ ]:
df.drop(null_value_Feat.index,axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

**Converting Categorical feature into numerical which is Actually Numerical**

In [ ]:
df['totals_visits']=df['totals_visits'].astype(int)
df['totals_hits']=df['totals_hits'].astype(int)
#df['totals_pageviews']=df['totals_pageviews'].astype(int)
#df['totals_newVisits']=df['totals_newVisits'].astype(int)
df['totals_transactionRevenue']=df['totals_transactionRevenue'].astype(int)

**Handling NaN values**

In [ ]:
df.isnull().sum()

In [ ]:
null_columns=df.columns[df.isnull().sum().values!=0]
df[null_columns].isnull().sum()

In [ ]:
df['totals_pageviews'].skew()

In [ ]:
sns.distplot(df['totals_pageviews'])

In [ ]:
df['totals_pageviews'].value_counts(normalize=True)*100

In [ ]:
mode=df['totals_pageviews'].mode()
mode

In [ ]:
#df['totals_pageviews'].fillna(mode,inplace=True)
df['totals_pageviews'].fillna(1, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.totals_newVisits.nunique()

**As all Contains same values we are going to drop this variable**

In [ ]:
df=df.drop('totals_newVisits',axis=1)

In [ ]:
df.isnull().sum().sum()

In [ ]:
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/Revenue_Without_null.csv')

**Split numerical and categorical features from the dataframe**

In [ ]:
df.info()

**As pageview is numerical we are convering into numerical**

In [ ]:
df['totals_pageviews']=df['totals_pageviews'].astype(int)

In [ ]:
num=df.select_dtypes(include=np.number).columns.tolist()
cat=df.select_dtypes(include=np.object).columns.tolist()
bool=df.select_dtypes(include=np.bool).columns.tolist()

In [ ]:
df.head(2)

**Handling Numerical values**

In [ ]:
for i in num:
  print('Unique Values in ',i,'is',df[i].nunique())

In [ ]:
df.totals_visits.nunique()

**We are dropping totals_visits as it contains only one value**

In [ ]:
df=df.drop('totals_visits',1)

**Doing Statistical Analysis between visit StartTime and visitId**

In [ ]:
df[['visitStartTime','visitId']]

In [ ]:
df['visitStartTime'].nunique()

In [ ]:
df['visitId'].nunique()

In [ ]:
mu1=df['visitStartTime'].mean()
mu1

In [ ]:
mu2=df['visitId'].mean()
mu2

In [ ]:
# df_visitStartTime=df['visitStartTime'].apply(lambda x: np.log(x+1))
# df_visitId=df['visitId'].apply(lambda x: np.log(x+1))

**Checking Normality Check Shapiro Test**

In [ ]:
from scipy.stats import shapiro
p_val1=shapiro(df['visitStartTime'][:5000])[1]

In [ ]:
p_val1

In [ ]:
p_val2=shapiro(df['visitId'][:5000])[1]
p_val2

In [ ]:
if p_val1<0.05:
    print('Data is not Normally Distributed')
else:
    print('Data is Normally Distributed')

In [ ]:
if p_val2<0.05:
    print('Data is not Normally Distributed')
else:
    print('Data is Normally Distributed')

In [ ]:
from scipy.stats import mannwhitneyu
p_val=mannwhitneyu(df['visitStartTime'][:5000],df['visitId'][:5000])[1]
p_val

In [ ]:
if p_val<0.05:
    print('The mean weight of the two sample is different, accept Ha')
else:
    print('The mean weight of the two sample is same, accept Ho')

**Since both Mean Weight are same hence we are dropping one variable**

In [ ]:
df.corrwith(df['totals_transactionRevenue'])

In [ ]:
df=df.drop('visitId',1)

**Handling visitStartTime by epoch & unix timestamp conversion and fetching date, Month and Year columns**

In [ ]:
df.visitStartTime

In [ ]:
visit_time=[]
fmt = "%Y-%m-%d"
for i in df.visitStartTime:
     t = datetime.datetime.fromtimestamp(int(i))
     visit_time.append(t.strftime(fmt))

In [ ]:
time_df=pd.DataFrame(visit_time,columns=['Full_date'])

In [ ]:
time_df.head(2)

In [ ]:
time_df['Visit_Year']=time_df['Full_date'].str.split('-').str[0].astype(int)
time_df['Visit_Month']=time_df['Full_date'].str.split('-').str[1].astype(int)
time_df['Visit_Date']=time_df['Full_date'].str.split('-').str[2].astype(int)

In [ ]:
time_df.head()

In [ ]:
time_df.info()

In [ ]:
time_df=time_df.drop('Full_date',1)

In [ ]:
df=pd.concat([df,time_df],axis=1)

In [ ]:
df=df.drop('visitStartTime',1)

In [ ]:
df.shape

In [ ]:
df.head(2)

**Handling Categorical values**

In [ ]:
for i in cat:
  print('Unique Values in ',i,'is',df[i].nunique())

**Dropping Categorical features which is having less than 10 unique values**

In [ ]:
for i in cat:
  if df[i].nunique()<=10:
    df=df.drop(i,axis=1)

In [ ]:
cat1=df.select_dtypes(include=np.object).columns.tolist()
for i in cat1:
  print('Unique Values in ',i,'is',df[i].nunique())

**sessionId contains lots of Unique values so we can drop it**

In [ ]:
df=df.drop('sessionId',axis=1)

In [ ]:
df.head(2)

In [ ]:
df.shape

**Handling date by epoch & unix timestamp conversion and fetching date, Month and Year columns but Year Month and date contains only same values so we are fetching Hrs, mins and secs** 

In [ ]:
date=[]
fmt = "%H:%M:%S"
for i in df.date:
     t = datetime.datetime.fromtimestamp(int(i))
     date.append(t.strftime(fmt))

In [ ]:
date_df=pd.DataFrame(date,columns=['Full_date'])

In [ ]:
date_df.head(3)

In [ ]:
date_df['Hours']=date_df['Full_date'].str.split(':').str[0].astype(int)
date_df['Minutes']=date_df['Full_date'].str.split(':').str[1].astype(int)
date_df['Seconds']=date_df['Full_date'].str.split(':').str[2].astype(int)

In [ ]:
date_df=date_df.drop('Full_date',1)

In [ ]:
df=pd.concat([df,date_df],axis=1)

In [ ]:
df=df.drop('date',1)

In [ ]:
df1=df.copy()

In [ ]:
df=df1.copy()

**Handling Categorical values**

In [ ]:
cat2=df.select_dtypes(include=np.object).columns.tolist()
for i in cat2:
  print('Unique Values in ',i,'is',df[i].nunique())

In [ ]:
df.geoNetwork_networkDomain.value_counts()

In [ ]:
# df[df['geoNetwork_networkDomain']=='(not set)']='Missing'
# df[df['geoNetwork_networkDomain']=='unknown.unknown']='Missing'

df["geoNetwork_networkDomain"].replace({"(not set)": "Missing", "unknown.unknown": "Missing"}, inplace=True)

In [ ]:
df.geoNetwork_networkDomain.value_counts()

In [ ]:
df.geoNetwork_networkDomain.value_counts(normalize=True)*100

In [ ]:
df.geoNetwork_subContinent.value_counts(normalize=True)*100

In [ ]:
df.geoNetwork_region.value_counts(normalize=True)*100

In [ ]:
df.geoNetwork_metro.value_counts(normalize=True)*100

In [ ]:
df.geoNetwork_city.value_counts(normalize=True)*100

**Dropping [ 'geoNetwork_region' , 'geoNetwork_metro' , 'geoNetwork_city' ] these Features as they contain almost 50% of Missing values**

In [ ]:
df=df.drop(['geoNetwork_region','geoNetwork_metro','geoNetwork_city'],1)

**Frequency Encoding**

In [ ]:
# Handling geoNetwork_country, trafficSource_source, geoNetwork_networkDomain, device_browser

In [ ]:
df.geoNetwork_country.value_counts(normalize=True)*100

In [ ]:
df.trafficSource_source.value_counts(normalize=True)*100

In [ ]:
df.columns

In [ ]:
# trafficSource_source
traffix_src=df.groupby('trafficSource_source')['totals_transactionRevenue'].sum()
traffix_src

In [ ]:
df['trafficSource_source']=df['trafficSource_source'].map(traffix_src)

In [ ]:
df.trafficSource_source.value_counts(normalize=True)*100

In [ ]:
# geoNetwork_country
geo_country=df.groupby('geoNetwork_country')['totals_transactionRevenue'].sum()
geo_country.values

In [ ]:
df['geoNetwork_country']=df['geoNetwork_country'].map(geo_country)

In [ ]:
df.geoNetwork_country.value_counts(normalize=True)*100

In [ ]:
# geoNetwork_networkDomain
geo_domain=df.groupby('geoNetwork_networkDomain')['totals_transactionRevenue'].sum()
geo_domain.values

In [ ]:
df['geoNetwork_networkDomain']=df['geoNetwork_networkDomain'].map(geo_domain)

In [ ]:
df.geoNetwork_networkDomain.value_counts(normalize=True)*100

In [ ]:
# device_browser
dev_browser=df.groupby('device_browser')['totals_transactionRevenue'].sum()
dev_browser

In [ ]:
df['device_browser']=df['device_browser'].map(dev_browser)

In [ ]:
df.device_browser.value_counts(normalize=True)*100

**Mapping geoNetwork_subContinent with 7 Continents { Africa, Antarctica, Asia, Australia/Oceania, Europe, North America, and South America }**



In [ ]:
df.geoNetwork_subContinent.value_counts(normalize=True)*100

In [ ]:
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Asia.*$)', "Asia")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Europe.*$)', "Europe")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Australasia.*$)', "Australia")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Africa.*$)', "Africa")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Melanesia.*$)', "Pacific")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Micronesian.*$)', "Pacific")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Polynesia.*$)', "Pacific")
df["geoNetwork_subContinent"]=df["geoNetwork_subContinent"].str.replace(r'(^.*Caribbean.*$)', "South America")

In [ ]:
df["geoNetwork_subContinent"]

In [ ]:
df["geoNetwork_subContinent"].value_counts(normalize=True)

In [ ]:
df = df[df.geoNetwork_subContinent != '(not set)']

In [ ]:
df["geoNetwork_subContinent"].value_counts(normalize=True)

In [ ]:
df.Visit_Year.nunique()

In [ ]:
df.Visit_Month.nunique()

In [ ]:
df.Visit_Date.nunique()

In [ ]:
df.Hours.nunique()

In [ ]:
df.Minutes.nunique()

In [ ]:
df.Seconds.nunique()

**Handling geoNetwork_networkDomain by Binary Encoding**




In [ ]:
# print('Feature device_operatingSystem has ',df.device_operatingSystem.nunique(),' Unique Values')
# print('Feature geoNetwork_subContinent has ',df.geoNetwork_subContinent.nunique(),' Unique Values')
# print('Feature device_isMobile has ',df.device_isMobile.nunique(),' Unique Values')

In [ ]:
# pip install --upgrade category_encoders

In [ ]:
# import category_encoders as ce

In [ ]:
# encoder= ce.BinaryEncoder(cols=['geoNetwork_networkDomain'])

In [ ]:
# df=encoder.fit_transform(df) 
# df

In [ ]:
#df.geoNetwork_networkDomain.value_counts()

In [ ]:
df_fullVisitorId=df.fullVisitorId

In [ ]:
df=df.drop('fullVisitorId',1)

In [ ]:
df

In [ ]:
df.select_dtypes(include=np.object)

**Doing OHE with the Remaining Categorical Columns**

In [ ]:
# print(df.geoNetwork_networkDomain.nunique())
print('Feature device_operatingSystem has ',df.device_operatingSystem.nunique(),' Unique Values')
print('Feature geoNetwork_subContinent has ',df.geoNetwork_subContinent.nunique(),' Unique Values')
print('Feature device_isMobile has ',df.device_isMobile.nunique(),' Unique Values')

In [ ]:
ohe_df=pd.get_dummies(df[['device_operatingSystem','geoNetwork_subContinent','device_isMobile']],drop_first=True)

In [ ]:
ohe_df

In [ ]:
df=pd.concat([df,ohe_df],1)

In [ ]:
df

In [ ]:
df=df.drop(['device_browser','device_operatingSystem','geoNetwork_subContinent','device_isMobile'],1)

In [ ]:
df

In [ ]:
bool

**Checking Outliers**

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.dtypes.value_counts()

In [ ]:
for i in df.columns:
  if df[i].dtypes=='int64':
    sns.boxplot(df[i])
    plt.show()

**These Three columns have OUTLIERS [ visitNumber , totals_hits, totals_page_views ]**

In [ ]:
outliers=['visitNumber', 'totals_hits', 'totals_pageviews']
for i in outliers:
  sns.boxplot(df[i])
  plt.show()

In [ ]:
sns.distplot(df.totals_pageviews,kde=False)

**Doing Power Transformation for outliers Data**

In [ ]:
# from sklearn.preprocessing import PowerTransformer
# pt = PowerTransformer()
# pt.fit([df.totals_pageviews])

In [ ]:
# pt_df=pt.transform([df.totals_pageviews])

In [ ]:
# sns.distplot(pt_df)

In [ ]:
# sns.boxplot(pt_df)

**Log Transformation**

In [ ]:
outliers=['visitNumber', 'totals_hits', 'totals_pageviews']


log_visit_no=df.visitNumber.apply(lambda x:np.log(x))
log_tot_hits=df.totals_hits.apply(lambda x:np.log(x))
log_tot_views=df.totals_pageviews.apply(lambda x:np.log(x))

In [ ]:
sns.distplot(log_tot_views)
print('Skewness : ',log_tot_views.skew())

In [ ]:
sns.distplot(log_tot_hits)
print('Skewness : ',log_tot_hits.skew())

In [ ]:
sns.distplot(log_visit_no)
print('Skewness : ',log_visit_no.skew())

**BoxCox Transformation**

In [ ]:
import scipy.stats as stats
trans1,lam=stats.boxcox(df['visitNumber'])
trans2,lam=stats.boxcox(df['totals_hits'])
trans3,lam=stats.boxcox(df['totals_pageviews'])

In [ ]:
sns.distplot(trans1)
print('Skewness : ',pd.DataFrame(trans1).skew().values)
plt.show()

In [ ]:
sns.distplot(trans2)
print('Skewness : ',pd.DataFrame(trans2).skew().values)
plt.show()

In [ ]:
sns.distplot(trans3)
print('Skewness : ',pd.DataFrame(trans3).skew().values)
plt.show()

**BOXCOX Transformation is performing better than LOG Transformation**

**Creating Trans DataFrame to append all BoxCox Transformed Features**

In [ ]:
trans=pd.DataFrame(trans1,columns=['visitNumber'])

In [ ]:
#trans['visitNumber']=pd.DataFrame(trans1)
trans['totals_hits']=pd.DataFrame(trans2)
trans['totals_pageviews']=pd.DataFrame(trans3)

In [ ]:
trans.reset_index(drop=True, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
trans.head()

In [ ]:
trans

In [ ]:
df=df.drop(['visitNumber','totals_hits','totals_pageviews'],1)

**Concating Tranformed features with the Dataframe**

In [ ]:
df=pd.concat([df,trans],axis=1)

**Base Model**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
x=df.drop('totals_transactionRevenue',1)
y=df['totals_transactionRevenue']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=10,stratify=y)

In [ ]:
mod_logit=LogisticRegression(max_iter=2000)
mod_logit.fit(x_train,y_train)

In [ ]:
# dt=DecisionTreeClassifier()
# dt.fit(x_train,y_train)

In [ ]:
# y_pred=dt.predict(x_test)
y_pred=mod_logit.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print('       Actual       ')
print('    ',0,'     ',1)
print(confusion_matrix(y_test,y_pred))

In [ ]:
485/(485+2968)

**Feature Importance**

In [ ]:
# Checking Feature Importance Using Decision tree

In [ ]:
x=df.drop('totals_transactionRevenue',1)
y=df['totals_transactionRevenue']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=10,stratify=y)

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)

In [ ]:
y_pred=dt.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
# pd.Series(dt.feature_importances_, index=x_train.columns).plot(kind='barh')

In [ ]:
# nlargest is used only for Series
# pd.DataFrame(feat_importances,index=x_test.columns).plot(kind='barh')

In [ ]:
#feat_importances.plot(kind='barh')

plt.figure(figsize=(20,10))
(pd.Series(dt.feature_importances_, index=x_train.columns)
   .nlargest(50)
   .plot(kind='barh'))

In [ ]:
pd.Series(dt.feature_importances_, index=x_train.columns).nlargest(18).index

#  Checking Mullticollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(df.values,i) for i in range(df.shape[1])]
vif['feature']=df.columns
vif.sort_values('VIF',ascending=False)

In [ ]:
# df=df.drop('Hours',1)
# df=df.drop('Visit_Year',1)
# df=df.drop('totals_hits',1)
# df=df.drop('geoNetwork_subContinent_Northern America',1) 
df=df.drop('device_operatingSystem_Windows',1)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
# Checking Model Performance After Removing Multicollinearity

In [ ]:
x=df.drop('totals_transactionRevenue',1)
y=df['totals_transactionRevenue']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=10,stratify=y)

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)

In [ ]:
y_pred=dt.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
plt.figure(figsize=(20,10))
(pd.Series(dt.feature_importances_, index=x_train.columns)
   .nlargest(50)
   .plot(kind='barh'))

In [ ]:
selected_feat=pd.Series(dt.feature_importances_, index=x_train.columns).nlargest(14).index
selected_feat

In [ ]:
df_imp=df[selected_feat]

In [ ]:
df_imp

# Handling Imbalance Problem

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
recall_score(y_test,y_pred)

In [ ]:
df.totals_transactionRevenue.value_counts()

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import NearMiss

In [ ]:
class_0=500000
class_1=100000

In [ ]:
# count_class_0 = 2000
# count_class_1 = 2000
# pipe = make_pipeline(SMOTE(sampling_strategy={1: count_class_1})
#                     ,NearMiss(sampling_strategy={0: count_class_0}))

# x_res, y_res = pipe.fit_resample(X, Y)

In [ ]:
x=df_imp
y=df['totals_transactionRevenue']

In [ ]:
pipe=make_pipeline(SMOTE(sampling_strategy={1:class_1}),
                         NearMiss(sampling_strategy={0:class_0}))

In [ ]:
# The Output will be in array
x_bal, y_bal = pipe.fit_resample(x, y)

In [ ]:
x_bal.shape

In [ ]:
y_bal.shape

In [ ]:
x_bal_df=pd.DataFrame(x_bal,columns=x.columns)
# y can be array

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x_bal_df,y_bal,test_size=0.3,random_state=10,stratify=y_bal)

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)

In [ ]:
y_pred=dt.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

**Checking with Multiple Models**

In [ ]:
x_bal_df.head(3)

In [ ]:
y_bal

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score,KFold

In [ ]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
clf1 = DecisionTreeClassifier(random_state=0)

In [ ]:
bagging1 = BaggingClassifier(base_estimator=clf1, n_estimators=21, max_samples=0.8,oob_score=True, random_state=0)
#bagging2 = BaggingClassifier(base_estimator=clf2, n_estimators=11, max_samples=0.8,oob_score=True, random_state=0)

In [ ]:
from sklearn.metrics import make_scorer,recall_score

scorer=make_scorer(recall_score,average='weighted')

score1 = cross_val_score(clf1, x_bal_df, y_bal, cv=kf, scoring=scorer)
score2 = cross_val_score(bagging1,x_bal_df, y_bal, cv=kf, scoring=scorer)
#score3 = cross_val_score(bagging2,x_bal_df, y_bal, cv=kf, scoring=scorer)

In [ ]:
np.mean(score1), np.std(score1)

In [ ]:
np.mean(score2), np.std(score2)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
clf1 = RandomForestClassifier(random_state=0)
clf2 = XGBClassifier(random_state=0)

In [ ]:
for i,j in zip([clf1,clf2],['RF','XG']):
    scorer=make_scorer(recall_score,average='weighted')
    score_vot = cross_val_score(i, x_bal_df, y_bal, cv=kf, scoring=scorer)
    print(j,np.mean(score_vot),np.std(score_vot))

**HyperParameter Tuning**

In [ ]:
x_bal_df=pd.DataFrame(x_bal,columns=x.columns)
y_bal

In [ ]:
# Parameter Tuning
be=[]
ve=[]
for i in [5,10,20,40,60,80,100,120,150,200,250,300,400]:
    #for j in [10,15,20,30]:
    rf=RandomForestClassifier(n_estimators=i,random_state=48)
    #scorer = make_scorer(f1_score, average = 'weighted')
    score=cross_val_score(rf,x_bal_df,y_bal,cv=kf,scoring='recall')
    print('n_estimator:',i)
    print('Bias Error',1-np.mean(score))
    print('Variance Error',np.std(score)/np.mean(score))
    be.append(1-np.mean(score))
    ve.append(np.std(score)/np.mean(score))

In [ ]:
# from sklearn.model_selection import GridSearchCV

# params={#'n_estimators':[50,80,100,120,140,190,200,210],
#         'max_features':['auto','sqrt','log2'],
#         'criterion':['gini','entropy'],
#         'max_depth':[10,15,20,25,30]}

In [ ]:
# rf=RandomForestClassifier()
# hyp_mod=GridSearchCV(rf,param_grid=params,scoring='recall')

In [ ]:
# tun_mod=hyp_mod.fit(x_bal_df, y_bal)

In [ ]:
# tun_mod.best_params_

In [ ]:
# Best Parameters => {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt'}

In [ ]:
# tun_mod.best_score_

**PCA**

In [ ]:
x_bal_df=pd.DataFrame(x_bal,columns=x.columns)
y_bal

In [ ]:
x=x_bal_df
y=y_bal

In [ ]:
x.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
inp_sc=sc.fit_transform(x)
inp_sc=pd.DataFrame(inp_sc,columns=x.columns)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca_mod=PCA(x.shape[1])
pcadata=pca_mod.fit_transform(inp_sc)

In [ ]:
pcadata=pd.DataFrame(pcadata,columns=['PC1','PC2','PC3','PC4',
                                      'PC5','PC6','PC7','PC8',
                                      'PC9','PC10','PC11','PC12',
                                      'PC13','PC14'])

In [ ]:
exp_var=pca_mod.explained_variance_ratio_

In [ ]:
cum_var=exp_var[0]
itr=2
for j in exp_var[1:]:
    cum_var=cum_var+j
    if cum_var>0.95:
        break
    itr=itr+1

print('The 95 percent of varaince is explained by',itr, ' Principle components')

In [ ]:
pd.DataFrame(exp_var).plot(kind='bar')

In [ ]:
pcadata1=pcadata.iloc[:,:12]

In [ ]:
pcadata1.head(2)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=48)
xtrain1,xtest1,ytrain1,ytest1=train_test_split(pcadata1,y,test_size=0.3,random_state=48)

In [ ]:
mod=RandomForestClassifier(random_state=0)
mod.fit(xtrain,ytrain)
ypred=mod.predict(xtest)
print("Without PCA",recall_score(ytest,ypred))

In [ ]:
mod1=RandomForestClassifier(random_state=0)
mod1.fit(xtrain1,ytrain1)
ypred1=mod1.predict(xtest1)
print("With PCA",recall_score(ytest1,ypred1))

In [ ]:
x_bal_df.shape

In [ ]:
# Final Model

In [ ]:
x=x_bal_df
y=y_bal

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=48,stratify=y)
xtrain1,xtest1,ytrain1,ytest1=train_test_split(pcadata1,y,test_size=0.3,random_state=48,stratify=y)

In [ ]:
# mod=RandomForestClassifier(**,random_state=0)
# Best Parameters => {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt'}
mod_final=RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=20, max_features='sqrt', random_state=0)
mod_final.fit(xtrain,ytrain)
ypred=mod_final.predict(xtest)

In [ ]:
print("Recall Score without PCA",recall_score(ytest1,ypred))

In [ ]:
mod_pca_final=RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=20, max_features='sqrt', random_state=0)
mod_pca_final.fit(xtrain1,ytrain1)
ypred=mod_pca_final.predict(xtest1)

In [ ]:
print("Recall Score With PCA",recall_score(ytest1,ypred))

In [ ]:
print(confusion_matrix(ytest1,ypred))